### Задание 2 (3 балла)
Обучите 1 word2vec и 1 fastext модель в gensim. В каждой из модели нужно задать все параметры, которые мы разбирали на семинаре. Заданные значения должны отличаться от дефолтных и от тех, что мы использовали на семинаре.

In [46]:
import tensorflow as tf
import numpy as np
import pandas as pd
from string import punctuation
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_distances
import gensim

In [2]:
wiki = open('wiki_data.txt', encoding='utf8').read().split('\n')

In [3]:
def preprocess(text):
    tokens = text.lower().split()
    tokens = [token.strip(punctuation) for token in tokens]
    return tokens

In [4]:
texts = [preprocess(text) for text in wiki]

In [21]:
%%time
w2v = gensim.models.Word2Vec(texts, 
                             vector_size=400, 
                             min_count=50, 
                             max_vocab_size=20000,
                             window=4,
                             epochs=10,
                             sg=1, 
                             hs=0,
                             negative=12,
                             sample=1e-4,
                             ns_exponent=0.6)

Wall time: 1min 2s


In [26]:
w2v.wv.most_similar('машина')

[('двигателем', 0.5316390991210938),
 ('шасси', 0.5194934606552124),
 ('двигатель', 0.4993988275527954),
 ('модель', 0.4937862157821655),
 ('мощность', 0.48875167965888977),
 ('подвеска', 0.483656644821167),
 ('машину', 0.46757638454437256),
 ('автомобиль', 0.4543384611606598),
 ('колёс', 0.4447077810764313),
 ('автомобиля', 0.43612030148506165)]

In [36]:
%%time
ft = gensim.models.FastText(texts, 
                            vector_size=500, 
                            min_count=50, 
                            max_vocab_size=20000,
                            window=6,
                            epochs=10, 
                            hs=1,
                            cbow_mean=1,
                            min_n=4, 
                            max_n=7)

Wall time: 3min 9s


In [37]:
ft.wv.most_similar('машина')

[('машину', 0.6642086505889893),
 ('машин', 0.6501579284667969),
 ('машины', 0.6370182633399963),
 ('модель', 0.44380733370780945),
 ('мощность', 0.4420733153820038),
 ('автомобиля', 0.4212818145751953),
 ('подвеска', 0.4182339608669281),
 ('двигателем', 0.4160038232803345),
 ('шасси', 0.4043676257133484),
 ('версия', 0.3960721790790558)]

### Задание 1 (3 балла)
Обучите word2vec модели с негативным семплированием (cbow и skip-gram) с помощью tensorflow аналогично тому, как это было сделано в семинаре. Вам нужно изменить следующие пункты: 1) добавьте лемматизацию в предобработку (любым способом)
2) измените размер окна на 6 для cbow и 12 для skip gram (обратите внимание, что размер окна = #слов слева + #слов справа, в gen_batches в семинаре window не так используется)
3) измените часть с np.random.randint(vocab_size) так, чтобы случайные негативные примеры выбирались обратно пропорционально частотностям слов (частотные должны выбираться реже, а редкие чаще)

Выберете несколько не похожих по смыслу слов, и протестируйте полученные эмбединги (найдите ближайшие слова и оцените правильность, как в семинаре)

In [53]:
wiki = open('wiki_data.txt', encoding='utf8').read()

In [52]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [ ]:
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from string import punctuation

In [3]:
def preprocess(text):
    tokens = text.lower().split()
    tokens = [token.strip(punctuation) for token in tokens]
    return tokens

In [ ]:
tokens = preprocess(wiki)
words = ', '.join(tokens)

In [12]:
words_analized = [morph.parse(token) for token in words]

In [ ]:
morph.parse(tokens)[0].normal_form